In [1]:
import pandas as pd 
import numpy as np
import itertools 
import requests
import ortools
import os

In [2]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [3]:
dt_gx = pd.read_excel('广西.xlsx',
                      dtype={'门店编号':str})
#df_gx = pd.DataFrame.from_dict(dt_gx,orient='index')
#dt_gx['门店编号'][0]


In [4]:
dt_gx.tail()

,门店名称,门店编号,门店地址,求和项:订货数量
25,小伍彩票店,450107100784,广西壮族自治区南宁市西乡塘区广西省南宁市衡阳西路38一15号,23
26,兴宁中鼎城市锦绣花园店,50220582,广西壮族自治区南宁市兴宁区唐山路46栋中鼎城市花园12楼1单元12111号房,7
27,兴盛优选劳燕华店,450107101438,广西壮族自治区南宁市西乡塘区北大北路25号梦泽园小区老区西侧旁66号商铺,11
28,亿鼎明都锦绣花园水果派店,450107100782,广西壮族自治区南宁市西乡塘区唐山路54号明都锦绣花园5栋2304,4
29,中华苑小区菜鸟驿站,450102100624,广西壮族自治区南宁市兴宁区中华路22号中华苑小区1号楼第11号车库,6


In [5]:
import pymysql
config = {
    'host': '192.168.1.227',
    'port': 4000,
    'user': 'liyufan',
    'passwd': 'Juno#2021',
    'charset':'utf8mb4',
    'cursorclass':pymysql.cursors.DictCursor
    }
conn = pymysql.connect(**config)

In [6]:
def sub_task(cal_df):
        requests.adapters.DEFAULT_RETRIES = 5
        s = requests.session()
        s.keep_alive = False
        cal_df['navi_distance'] = cal_df.apply(
            lambda df: geo_location_distance(df['lon0'], df['lat0'], df['lon1'], df['lat1']), axis=1)
        return cal_df.copy()

In [7]:
def geo_location_distance(o_X, o_Y, d_X, d_Y):
#     '''
#     根据经纬度调用高德API获取导航距离（使用距离最短策略:strategy=2）
#     :param o_X:起点经度
#     :param o_Y:起点纬度
#     :param d_X:终点经度
#     :param d_Y:终点纬度
#     :return:导航距离（km为单位）
#     '''
    k = 'c33b02b68bd7785d75ed05f0178a8e05'
    if d_X == 0 or d_Y == 0 or str(d_X).lower().strip() == 'nan':
        return 999
    origin = str(o_X) + ',' + str(o_Y)
    destination = str(d_X) + ',' + str(d_Y)

    #url = "https://restapi.amap.com/v3/direction/driving?key=%s&origin=%s&destination=%s&strategy=0&extensions=base" % (
        #k, origin, destination)

    url = "https://restapi.amap.com/v3/distance?key=%s&origins=%s&destination=%s&type=1" % (
        k, origin, destination)

    headers = {'Connection': 'close', }

    try:
        data = requests.get(url, headers=headers)
        contest = data.json()
        if contest['status'] == '1':
            # route = contest['route']
            # result = route['paths'][0]
            # out = float(result['distance']) / 1000
            result = contest['results'][0]
            out = float(result['distance']) / 1000
        else:
            print(contest['info'])
            print(d_Y)
            out = 999
    except:
        out = 999
    return out

In [8]:
def get_route_total_distance(store_table,store_distance_df):
    #调用or tools, assign origin and vehicle 
    manager = pywrapcp.RoutingIndexManager(store_table.shape[0],
                                       1, 0)
    routing = pywrapcp.RoutingModel(manager)
    
    def distance_callback(from_index, to_index):
#         """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)

        #只计算单程线路，不计算返程，将所有目的地为仓库的距离排除计算外
        if to_node == 0:
            return 0
    
#     from_node = from_index
#     to_node = to_index
        from_code = store_table.loc[int(from_node),'门店编号']
        to_code = store_table.loc[int(to_node),'门店编号']
    
#     print(from_code)
#     print(to_code)
    
    
        if from_code > to_code:
            return store_distance_df[
            (store_distance_df['village_code_0'] == to_code) & (store_distance_df['village_code_1'] == from_code)][
            'navi_distance'].values[0]
        elif from_code < to_code:
            return store_distance_df[
            (store_distance_df['village_code_0'] == from_code) & (store_distance_df['village_code_1'] == to_code)][
            'navi_distance'].values[0]
        else:
            return 0

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    solution = routing.SolveWithParameters(search_parameters)
    total_distance = 0
    route_order = ''
    
    def print_solution(manager, routing, solution):
        t_d = solution.ObjectiveValue()
        r_o =[]
        index = routing.Start(0)
        while not routing.IsEnd(index):
            r_o.append(store_table.loc[int(index),'门店编号'])
            index = solution.Value(routing.NextVar(index))
            
        return t_d,','.join(r_o)
    
    if solution:
        total_distance,route_order = print_solution(manager, routing, solution)
    return total_distance,route_order

In [10]:
temp_df.head()

0            白苍岭小区奶屋
1          北湖建兴园菜鸟驿站
2              博白白切饭
3    菜鸟驿站唐山路54号103铺店
4              丹群杂货店
Name: 门店名称, dtype: object

In [10]:
#创建距离，线路dict
distance_dict = {}
route_dict = {}
#遍历key value的值
#for key,value in dt_gx.items():
temp_df = dt_gx.copy()
store_info = pd.read_sql("select storeCode,mapX, mapY from xsyx_frxs_base.t_store where storeCode in "+str(tuple(temp_df['门店编号'].values.tolist())),conn)
temp_df['lat'] = store_info['mapY']
temp_df['lng'] = store_info['mapX']  
    
    #提取门店信息和经纬度
st_df = temp_df[['门店编号','lng','lat']].copy()
    
    #去除无经纬度的门店
if st_df['lng'].isnull().any():
    continue
    
    #添加仓库地址
st_df.loc[-1] = ['0', 114.606, 37.9361]  # adding a row
st_df.index = st_df.index + 1  # shifting index
st_df.sort_index(inplace=True)
    #value to list for use itertools.combination
lnglat = st_df.values.tolist()
dist_combination = list(itertools.combinations(lnglat, 2))
    
    #路径编码排序 for storage in database
reordered_list = []
for i in dist_combination:
    temp_list = []
    index = ''
    if str(i[0][0]) > str(i[1][0]): 
        index = str(i[1][0])+'-'+str(i[0][0])
        temp_list.extend(i[1]+i[0])
    else:
        index = str(i[0][0])+'-'+str(i[1][0])
        temp_list.extend(i[0]+i[1])
    temp_list.append(index)
    reordered_list.append(temp_list)
    
    #apply 排序函数to dataframe 
cal_combination_df = pd.DataFrame(data=reordered_list,columns=['village_code_0','lon0','lat0','village_code_1','lon1','lat1','village_distance_index'])
    #将编码转成string
cal_combination_df['village_code_0'] = cal_combination_df['village_code_0'].astype(str)
cal_combination_df['village_code_1'] = cal_combination_df['village_code_1'].astype(str)
    #高德API调导航距离
cal_combination_df_return = sub_task(cal_combination_df)
    #将导航距离转成integer
cal_combination_df_return['navi_distance'] = cal_combination_df_return.apply(lambda df:int(df['navi_distance']),axis=1)
    
    #调用google or tools
d,r = get_route_total_distance(st_df,cal_combination_df_return)
    #
distance_dict[key] = d
route_dict[key] = r



SyntaxError: 'continue' not properly in loop (<ipython-input-10-73f7d3ffb67a>, line 19)

In [41]:
distance_df = pd.DataFrame.from_records(distance_dict, index=[0]) 

In [42]:
route_df = pd.DataFrame.from_records(route_dict, index=[0]) 

In [43]:
final_result = pd.concat([distance_df, route_df], ignore_index=True)

In [44]:
final_result.rename({0: '路程（km）', 1: '线路'}, inplace=True)

In [45]:
final_result.to_excel('河北线路.xlsx')

In [11]:
store_df = dt_gx[['门店编号','lng','lat']].copy()

In [8]:
def geo_location_distance(o_X, o_Y, d_X, d_Y):
#     '''
#     根据经纬度调用高德API获取导航距离（使用距离最短策略:strategy=2）
#     :param o_X:起点经度
#     :param o_Y:起点纬度
#     :param d_X:终点经度
#     :param d_Y:终点纬度
#     :return:导航距离（km为单位）
#     '''
    k = 'c33b02b68bd7785d75ed05f0178a8e05'
    if d_X == 0 or d_Y == 0 or str(d_X).lower().strip() == 'nan':
        return 999
    origin = str(o_X) + ',' + str(o_Y)
    destination = str(d_X) + ',' + str(d_Y)

    #url = "https://restapi.amap.com/v3/direction/driving?key=%s&origin=%s&destination=%s&strategy=0&extensions=base" % (
        #k, origin, destination)

    url = "https://restapi.amap.com/v3/distance?key=%s&origins=%s&destination=%s&type=1" % (
        k, origin, destination)

    headers = {'Connection': 'close', }

    try:
        data = requests.get(url, headers=headers)
        contest = data.json()
        if contest['status'] == '1':
            # route = contest['route']
            # result = route['paths'][0]
            # out = float(result['distance']) / 1000
            result = contest['results'][0]
            out = float(result['distance']) / 1000
        else:
            print(contest['info'])
            print(d_Y)
            out = 999
    except:
        out = 999
    return out

In [31]:
cal_combination_df = pd.DataFrame(data=reordered_list,columns=['village_code_0','lon0','lat0','village_code_1','lon1','lat1','village_distance_index'])

In [34]:
sub_task(cal_combination_df,0)

0 start!


### 路径规划

In [37]:
store_table = store_df.copy()
store_distance_df = gx_distance.copy()

In [38]:
store_table.head()

,门店编号,lng,lat
0,0,108.536000,22.808600
1,450107100141,108.321327,22.831444
2,450107101124,108.295914,22.828035
3,450107101506,108.300827,22.824060
4,450107101580,108.295853,22.825460


In [39]:
store_distance_df.head()

,Unnamed: 0,village_code_0,lon0,lat0,village_code_1,lon1,lat1,village_distance_index,navi_distance
0,0,0,108.536,22.8086,450107100141,108.321327,22.831444,0-450107100141,27.626
1,1,0,108.536,22.8086,450107101124,108.295914,22.828035,0-450107101124,28.676
2,2,0,108.536,22.8086,450107101506,108.300827,22.824060,0-450107101506,27.545
3,3,0,108.536,22.8086,450107101580,108.295853,22.825460,0-450107101580,27.591
4,4,0,108.536,22.8086,450107100762,108.317253,22.831120,0-450107100762,30.302


In [107]:
# Solve the problem.
solution = routing.SolveWithParameters(search_parameters)

In [108]:
def print_solution(manager, routing, solution):
    """Prints solution on console."""
    print('Objective: {} km'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Route distance: {}km\n'.format(route_distance)


In [109]:
 # Print solution on console.
if solution:
    print_solution(manager, routing, solution)

Objective: 35 km
Route for vehicle 0:
 0 -> 21 -> 3 -> 18 -> 23 -> 15 -> 13 -> 14 -> 7 -> 28 -> 29 -> 26 -> 6 -> 4 -> 25 -> 20 -> 8 -> 2 -> 30 -> 12 -> 11 -> 9 -> 10 -> 5 -> 22 -> 1 -> 27 -> 24 -> 19 -> 16 -> 17 -> 0



In [114]:
store_distance_df.head(2)

,Unnamed: 0,village_code_0,lon0,lat0,village_code_1,lon1,lat1,village_distance_index,navi_distance
0,0,0,108.536,22.8086,450107100141,108.321327,22.831444,0-450107100141,27.626
1,1,0,108.536,22.8086,450107101124,108.295914,22.828035,0-450107101124,28.676


In [112]:
get_route_total_distance(store_table,store_distance_df)

(35,
 '0,50221707,450107101506,450107100432,50220094,450107100846,450102100003,450107101088,450107100884,450107101438,450107100782,450107100784,450107101077,450107101580,50220620,50221004,450107100082,450107101124,450102100624,450107100245,450107101045,450107100592,450107100067,450107100762,50221117,450107100141,50220582,50221241,450107100398,450107101083,450107100308')

## 路径规划函数

In [9]:
def get_route_total_distance(store_table,store_distance_df):
    #调用or tools, assign origin and vehicle 
    manager = pywrapcp.RoutingIndexManager(store_table.shape[0],
                                       1, 0)
    routing = pywrapcp.RoutingModel(manager)
    
    def distance_callback(from_index, to_index):
#         """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)

        #只计算单程线路，不计算返程，将所有目的地为仓库的距离排除计算外
        if to_node == 0:
            return 0
    
#     from_node = from_index
#     to_node = to_index
        from_code = store_table.loc[int(from_node),'门店编号']
        to_code = store_table.loc[int(to_node),'门店编号']
    
#     print(from_code)
#     print(to_code)
    
    
        if from_code > to_code:
            return store_distance_df[
            (store_distance_df['village_code_0'] == to_code) & (store_distance_df['village_code_1'] == from_code)][
            'navi_distance'].values[0]
        elif from_code < to_code:
            return store_distance_df[
            (store_distance_df['village_code_0'] == from_code) & (store_distance_df['village_code_1'] == to_code)][
            'navi_distance'].values[0]
        else:
            return 0

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    solution = routing.SolveWithParameters(search_parameters)
    total_distance = 0
    route_order = ''
    
    def print_solution(manager, routing, solution):
        t_d = solution.ObjectiveValue()
        r_o =[]
        index = routing.Start(0)
        while not routing.IsEnd(index):
            r_o.append(store_table.loc[int(index),'门店编号'])
            index = solution.Value(routing.NextVar(index))
            
        return t_d,','.join(r_o)
    
    if solution:
        total_distance,route_order = print_solution(manager, routing, solution)
    return total_distance,route_order

In [ ]:
for route in route_list:
#     读取线路门店列表和距离矩阵.;
    store_table = store_df.copy()
    store_distance_df = gx_distance.copy()
#     定义manager
    manager = pywrapcp.RoutingIndexManager(store_table.shape[0],
                                       1, 0)
    routing = pywrapcp.RoutingModel(manager)
    

In [185]:
cal_combination_df_return.head()

,village_code_0,lon0,lat0,village_code_1,lon1,lat1,village_distance_index,navi_distance
0,0,108.536,22.8086,450107100697,108.307602,22.847441,0-450107100697,31
1,0,108.536,22.8086,450107100346,108.314674,22.843071,0-450107100346,29
2,0,108.536,22.8086,450107100260,108.307945,22.850910,0-450107100260,30
3,0,108.536,22.8086,450107100829,108.305450,22.841141,0-450107100829,31
4,0,108.536,22.8086,450107101133,108.309860,22.846415,0-450107101133,31
